In [3]:
import pandas as pd
import pulp

#データの取得
s_df = pd.read_csv('example_yokohama - 責任者_有給取得数リスト.csv')
h_df = pd.read_csv('example_yokohama - 希望休有給リスト.csv')
m_df = pd.read_csv('example_yokohama - 出社必須日リスト.csv')
w_pair_forall_df = pd.read_csv('example_yokohama - 従業員組み合わせリスト_全営業日.csv')
w_pair_fors_df = pd.read_csv('example_yokohama - 従業員組み合わせリスト_特定営業日.csv')
d_particular_df = pd.read_csv('example_yokohama - 混雑予想日_営業停止日リスト.csv')

#数理最適化モデルの定義
prob = pulp.LpProblem('ShiftschedulingProblem', pulp.LpMinimize)

#従業員のリスト
W = s_df['従業員の名前'].tolist()

#日数の特定
m_end = int(input('月末の日にちを入力してください>>'))
D = list(range(1,m_end + 1))

#混雑予想日リスト
L1 = [row.Lv1土日 for row in d_particular_df.itertuples()]
L2 = [row.Lv2 for row in d_particular_df.itertuples()]
L3 = [row.Lv3 for row in d_particular_df.itertuples()]

#営業停止日リスト
N_sale = [row.営業停止日 for row in d_particular_df.itertuples()]

#Lv2混雑予想日リストにLv1混雑予想日に入っていない土日を追加
H_identify = 7 - int(input('月の初めの曜日を入力してください\n月:1 火:2 水:3 木:4 金:5 土:6 日:7'))
for d in D:
    if d not in L1:
        if d % 7 == H_identify or d % 7 == (H_identify + 1) % 7:
            L2.append(d)

#従業員の出社必須日のリスト
A_must = [(row.従業員の名前,row.出社必須日) for row in m_df.itertuples()]
        
#従業員の有給と希望休のリスト
H_wanted = [(row.従業員の名前,row.希望休有給) for row in h_df.itertuples()]

#有給取得日数
H_paid = {row.従業員の名前:row.有給取得数 for row in s_df.itertuples()}

#有給取得者リスト
H_paid_get = []
for w in H_paid:
    if H_paid[w] != 0:
        H_paid_get.append(w)      

#必要休暇数
H_required = {w:h_rq + 9 for w, h_rq in H_paid.items()}

#管理者のリスト
M_workers = [row.従業員の名前 for row in s_df.itertuples() if row.責任者 == 1]

#従業員組み合わせリスト全営業日のリスト
WW = [(row.従業員1の名前,row.従業員の名前2) for row in w_pair_forall_df.iteruples()]

#従業員組み合わせリスト特定営業日のリスト
WW_forp = {row.営業日：(row.従業員1の名前,row.従業員の名前2) for row in w_pair_fors_df.iteruples()}

#必要従業員数
W_required = {d:0 for d in D}
for d in D:
    if d in L3:
        W_required[d] = 4
    elif d in L2:
        W_required[d] = 3    
    else:
        W_required[d] = 2

#変数の定義
WD = [(w,d) for w in W for d in D]
x = pulp.LpVariable.dicts('x',WD,cat='Binary')  

#制約式の定義
#休暇日数を守る
for w in W:
    prob += pulp.lpSum([x[w,d] for d in D]) == m_end - H_required[w]
    
#希望休を守る
for w,d in H_wanted:
    prob += x[w,d] == 0
    
#混雑予想Lvに応じて出勤人数を設定
for d in D:
    if d not in N_sale:
        prob += pulp.lpSum([x[w,d] for w in W]) <= W_required[d] + 1
        prob += pulp.lpSum([x[w,d] for w in W]) >= W_required[d]

#責任者は必ず一人以上
for d in D:
    if d not in N_sale:        
        prob += pulp.lpSum(x[w,d] for w in M_workers) >= 1    
        
#出社必須日は必ず出社
for w,d in A_must:
    prob += x[w,d] == 1

#特定のペアは同じ日に出勤
for w1,w2 in WW:
    for d in D:
        prob += x[w1,d] + x[w2,d] <= 1
        
#4連勤以上を作らない
for w in W:
    for d in D[0:-3]:
        prob += x[w,d] + x[w,d+1] + x[w,d+2] + x[w,d+3] <=3

#3連休以上を作らない(有給取得者は可能)
for w in W:
    if w not in H_paid_get:
        for d in D[0:-2]:
            prob += x[w,d] + x[w,d+1] + x[w,d+2] >= 1
            
#目的関数の定義
#月木の従業員数が2人以下の日数を最小
D_move = []
for d in D :
    if (d % 7 == H_identify + 2 or d % 7 == H_identify + 5) and d not in N_sale:
        D_move.append(d)       
M_error = []
for d in D_move:
    if sum(x[w,d] for w in W) <= 2:
        M_error.append(d)
prob += pulp.lpSum(len(M_error))      
#求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

#計算結果の表示
#各日にちに割り当てられている従業員のリストを辞書に格納
D2Ws = {}
for d in D:
    D2Ws[d] = [w for w in W if x[w,d].value() == 1]   
W2Cs = {}
for w in W:
    W2Cs[w] = [d for d in D if x[w,d].value() == 1]

print('=COUNTIF(B:C,$G2)')
    

df = pd.DataFrame.from_dict(W2Cs, orient='index')
df.to_excel('example_yokohama_for_workers.xlsx') 

月末の日にちを入力してください>> 31
月の初めの曜日を入力してください
月:1 火:2 水:3 木:4 金:5 土:6 日:7 7


[nan]
[2, 1, 7, 8, 14, 15, 21, 22, 28, 29]
[nan]
[18]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/riku/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l_/3g64jjzd66z7zg2sxhpd3fv80000gn/T/077758b41b144affacfe00a8bd336587-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/l_/3g64jjzd66z7zg2sxhpd3fv80000gn/T/077758b41b144affacfe00a8bd336587-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 229 COLUMNS
At line 1369 RHS
At line 1594 BOUNDS
At line 1720 ENDATA
Problem MODEL has 224 rows, 125 columns and 890 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 18 variables fixed
Cgl0003I 16 fixed, 0 tightened bounds, 11 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0004I processed model ha